In [1]:
library(dplyr)
library(ROCR)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




**V3 define ABPRS on AoU:**
$$
ABPRS = \beta^{AoU}\cdot\left[\beta^{UKBB}_0 \cdot PRS\right] + \beta^{AoU}_\theta\cdot\left[\sum_{i=1}^n \beta^{UKBB}_i \cdot f_{\theta}(SNP_i)\right]
$$

In [37]:
pheno <- "ldl"

# Functions

In [38]:
m_func <- function(data, race_num, df_coef){
    # filter race
    data <- data[data$race == race_num, ]
    
    y <- data$y
    X <- as.matrix(data[, -c(1,2,3)])
    
    df_X_names <- data.frame(var = colnames(X))
    df_coef <- inner_join(df_coef, df_X_names, by="var")

    X_all <- X[, df_coef$var]
    X_prs <- X_all[,1]
    X_theta <- X_all[,-c(1)]
    
    m1 <- X_prs * df_coef$coef[1]
    m2 <- X_theta %*% df_coef$coef[-1]
    
    return(list(m1, m2))
}

metric_func <- function(data, race_num, m1, m2){
    
    # discrete metric: AUC
    # continuous metric: R-squared
    data <- data[data$race == race_num, ]
    data$m1 <- m1
    data$m2 <- m2
    
    if (pheno %in% c("AD", "breast_cancer", "hypertension", "t2d")){
        mod1 <- glm(y ~ prs, data = data, family="binomial")
        mod2 <- glm(y ~ m1 + m2, data = data, family="binomial")
        
        prediction <- prediction(predict(mod1, newdata=data, type="response"), data$y)
        mod1_metric <- performance(prediction,measure="auc")@y.values[[1]]
        
        prediction <- prediction(predict(mod2, newdata=data, type="response"), data$y)
        mod2_metric <- performance(prediction,measure="auc")@y.values[[1]]
    } else {
        mod1 <- glm(y ~ prs, data = data, family="gaussian")
        mod2 <- glm(y ~ m1 + m2, data = data, family="gaussian")
        
        pred1 <- predict(mod1, newdata=data)
        mod1_metric <- cor(data$y, pred1)^2
        
        pred2 <- predict(mod2, newdata=data)
        mod2_metric <- cor(data$y, pred2)^2
    }
    df_rslt <- data.frame(external_PRS_metric=mod1_metric,
                          AB_PRS_metric=mod2_metric,
                          increase_percent=(mod2_metric-mod1_metric)/mod1_metric)
    return(df_rslt)
}
  

# Loading data

In [39]:
prs_dir <- paste0("/home/jupyter/workspaces/hypertensionukbb/AB_PRS_AoU/external_PRS/",pheno,"_external_PRS.csv")
data <- read.csv(prs_dir, header=T)
print(head(data))

      IID    prs
1 1000004 5.3444
2 1000033 5.4813
3 1000039 5.5607
4 1000042 5.0120
5 1000045 4.5311
6 1000059 4.8247


In [40]:
for (i in 1:22){
    print(paste("chr",i,"start..."))
    theta_dir <- paste0("/home/jupyter/workspaces/hypertensionukbb/AB_PRS_AoU/theta_SNP/",pheno,"_chr",i,"_theta")
    
    if (file.exists(theta_dir)){
        theta <- read.table(theta_dir, header=T)
        data <- cbind(data, theta)
    }
}

print(data[1:6, 1:5])

[1] "chr 1 start..."
[1] "chr 2 start..."
[1] "chr 3 start..."
[1] "chr 4 start..."
[1] "chr 5 start..."
[1] "chr 6 start..."
[1] "chr 7 start..."
[1] "chr 8 start..."
[1] "chr 9 start..."
[1] "chr 10 start..."
[1] "chr 11 start..."
[1] "chr 12 start..."
[1] "chr 13 start..."
[1] "chr 14 start..."
[1] "chr 15 start..."
[1] "chr 16 start..."
[1] "chr 17 start..."
[1] "chr 18 start..."
[1] "chr 19 start..."
[1] "chr 20 start..."
[1] "chr 21 start..."
[1] "chr 22 start..."
      IID    prs rs11591147   rs34791230 rs114952518
1 1000004 5.3444          0 -0.009249335           0
2 1000033 5.4813          0  0.000000000           0
3 1000039 5.5607          0 -0.009249335           0
4 1000042 5.0120          0  0.000000000           0
5 1000045 4.5311          0 -0.031017751           0
6 1000059 4.8247          0 -0.009249335           0


In [41]:
pheno_dir <- "/home/jupyter/workspaces/hypertensionukbb/AB_PRS_AoU/AoU_phenotype/phenotype.csv"

if (pheno == "breast_cancer"){
    cols_select <- c("IID", "gender", "race", pheno)
    df_pheno <- read.csv(pheno_dir, header=T) %>% select(any_of(cols_select))
    colnames(df_pheno) <- c("IID", "gender", "race", "y")
    print(dim(df_pheno))
    ## only select female
    df_pheno <- df_pheno[df_pheno$gender == 1,]
    print(dim(df_pheno))
    df_pheno <- df_pheno %>% select(any_of(c("IID", "race", "y")))
} else{
    cols_select <- c("IID", "race", pheno)
    df_pheno <- read.csv(pheno_dir, header=T) %>% select(any_of(cols_select))
    colnames(df_pheno) <- c("IID", "race", "y")
}
print("loading phenotype done.")

[1] "loading phenotype done."


In [42]:
## filter continuous phenotype
if (pheno %in% c("bmi", "cholesterol", "hdl", "ldl")){
    dir <- "/home/jupyter/workspaces/hypertensionukbb/AB_PRS_AoU/AoU_phenotype/continuous_phenotype_range.csv"
    df_range <- read.csv(dir, header=T)
    df_range <- df_range[df_range$phenotype == pheno,]
    print(df_range)
    # filter
    print(paste("number of participants, before filter:", nrow(df_pheno)))
    df_pheno <- df_pheno[(df_pheno$y >= df_range$min)&(df_pheno$y <= df_range$max),]
    print(paste("number of participants, after filter:", nrow(df_pheno)))
} else {
    print("discrete phenotype!")
}

  phenotype   min     max
4       ldl 4.788 176.346
[1] "number of participants, before filter: 245394"
[1] "number of participants, after filter: 100403"


In [43]:
## inner_join phenotype and genotype
data <- inner_join(df_pheno, data, by="IID")
print(dim(data))
print(data[1:6, 1:7])

[1] 100403    950
      IID race   y    prs rs11591147   rs34791230 rs114952518
1 1000004    2 135 5.3444          0 -0.009249335           0
2 1000042    1  83 5.0120          0  0.000000000           0
3 1000091    2 123 5.7225          0 -0.009249335           0
4 1000095    2  89 5.1961          0 -0.009249335           0
5 1000104    6  90 4.3759          0  0.000000000           0
6 1000109    6 159 5.0589          0  0.000000000           0


In [44]:
coef_dir <- paste0("/home/jupyter/workspaces/hypertensionukbb/AB_PRS_AoU/UKBB_glm_model/",pheno,"_glm_model.RData")
load(coef_dir)
print(head(coef_mod2))

          var       coef
1   intercept -0.4136903
2         prs  0.2258363
3  rs11591147  1.1872680
4  rs34791230  0.8282514
5 rs114952518  0.3667169
6  rs72660960  0.1822667


# Fit AB-PRS model

In [45]:
race_vec <- c("Black", "White", "Asian", "Hawaiian_Islander", "Mid_Eest_North_Africa", "other")
df_race <- data.frame(Race=race_vec)

rslt <- m_func(data, 1, coef_mod2)
df_rslt <- metric_func(data, 1, rslt[[1]], rslt[[2]])
for (i in 2:6){
    rslt <- m_func(data, i, coef_mod2)
    temp <- metric_func(data, i, rslt[[1]], rslt[[2]])
    df_rslt <- rbind(df_rslt, temp)
}
df_rslt <- cbind(df_race, df_rslt)

df_rslt
rslt_dir <- paste0("/home/jupyter/workspaces/hypertensionukbb/AB_PRS_AoU/AB_PRS_result/",pheno,"_V3_result.csv")
write.csv(df_rslt, rslt_dir, row.names = FALSE)

Race,external_PRS_metric,AB_PRS_metric,increase_percent
<chr>,<dbl>,<dbl>,<dbl>
Black,9.723439e-04,0.01574373,15.19152322
White,2.720779e-03,0.02375494,7.73093201
Asian,2.589643e-04,0.02524544,96.48616292
Hawaiian_Islander,5.294342e-02,0.05791847,0.09396914
Mid_Eest_North_Africa,3.215415e-05,0.02440627,758.03946568
other,8.927539e-05,0.01713982,190.98819679
